In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torch.autograd import Variable, Function
from torch.nn.parameter import Parameter
import random

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from numpy.linalg import solve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

%matplotlib inline

# Prepare the data:

!curl -O http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip
!cd ml-100k/
!rm -r ml-100k.zip

In [3]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = 1.0#row[3]
ratings

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [5]:
print(str(n_users) + ' users')
print(str(n_items) + ' items')
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.1f}%'.format(sparsity))

943 users
1682 items
Sparsity:  6.3%


### Make train/test split:

In [6]:
def train_test_split(ratings, size=1):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=size, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [7]:
train_data, test_data = train_test_split(ratings, size=1)

### Make 1000 test items for HR@N:

In [7]:
def dict_test(train_data, test_data):
    nonzero_col_idx = test_data.argmax(axis=1)
    n_users = train_data.shape[0]
    dt = dict()
    for user in range(n_users):
        row = train_data[user]
        mask1 = row.argsort()[::-1]
        mask2 = row[mask1] < 1.0
        list_idx = list(mask1[mask2])
        list_idx.remove(nonzero_col_idx[user])
        list_idx = list_idx[:999]
        list_idx.append(nonzero_col_idx[user])
        dt[user] = np.array(list_idx)
    return dt    

In [8]:
test_dict = dict_test(train_data, test_data)

### Create indices/data Dataloader:

In [9]:
class MovieLens100k(Dataset):
    def __init__(self, train_data):
        rows, cols = train_data.nonzero()
        self.rows = rows
        self.cols = cols
        self.data = train_data

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
        return self.rows[idx], self.cols[idx]

In [10]:
train_dt = MovieLens100k(train_data)
test_dt = MovieLens100k(test_data)

In [11]:
train_dataloader = DataLoader(train_dt, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dt, batch_size=64, shuffle=True)

### HR function:

In [12]:
def hr(test_data, test_dict, rating, hm=10):
    
    test_indices = test_data.nonzero()[1]
    total = test_data.shape[0]
    hit = 0.0
    for i in np.arange(total):
        check_indices = np.argsort(rating[i][test_dict[i]])[::-1][:hm]
        if test_indices[i] in set(test_dict[i][check_indices]):
            hit += 1.0
    return hit / total    

# LightFM WARP:

In [13]:
# Instantiate and train the model
model = LightFM(
    no_components=50,
    k=5,
    n=10,
    learning_schedule='adagrad',
    loss='warp',
    learning_rate=0.005,
    rho=0.95,
    epsilon=1e-06,
    item_alpha=0.5,
    user_alpha=0.3,
    max_sampled=15,
    random_state=13,
)

In [14]:
model.fit(csr_matrix(train_data), epochs=200, num_threads=2, verbose=True)

Epoch: 100% 200/200 [00:25<00:00,  7.77it/s]


In [19]:
ub, wu = model.get_user_representations()
ib, wi = model.get_item_representations()
w = (wu @ wi.T) + ub.T[:, np.newaxis] + ib
w

array([[-0.19865036, -0.5474862 , -0.6383625 , ..., -0.9256259 ,
        -0.9274133 , -0.924332  ],
       [ 0.18439803, -0.16417046, -0.25517118, ..., -0.541932  ,
        -0.5441742 , -0.5411384 ],
       [ 0.20720148, -0.14136712, -0.23236978, ..., -0.5191307 ,
        -0.5213729 , -0.51833516],
       ...,
       [ 0.32412323, -0.0244507 , -0.11545103, ..., -0.40220818,
        -0.40445364, -0.40141708],
       [ 0.13970917, -0.20885937, -0.29985982, ..., -0.5866214 ,
        -0.588863  , -0.585827  ],
       [-0.04027218, -0.38884032, -0.47984084, ..., -0.7666024 ,
        -0.768844  , -0.765808  ]], dtype=float32)

In [20]:
hr(test_data, test_dict, w, hm=10)

0.15270413573700956

# PyTorch WARP:

## Define WARP Loss:

In [21]:
class WARP(Function): 
    '''
    autograd function of WARP loss
    '''
    @staticmethod
    def forward(ctx, input, target, max_num_trials=None):
        
        batch_size = target.size()[0]
        if max_num_trials is None: 
            max_num_trials = target.size()[1] - 1
        
        positive_indices = torch.zeros(input.size(), device=device)
        negative_indices = torch.zeros(input.size(), device=device)
        L = torch.zeros(input.size()[0], device=device)
        
        all_labels_idx = np.arange(target.size()[1])
        
        Y = float(target.size()[1])
        J = torch.nonzero(target)
        
        for i in range(batch_size): 
            
            msk = np.ones(target.size()[1], dtype = bool)
            
            # For i user in batch choose ONE item!
            msk_J = J[:, 0] == i
            indice = np.random.choice(np.arange(J[msk_J].shape[0]), 1)
            J[msk_J][indice].squeeze()[1]
            
            # Find the positive label for this example
            j = J[msk_J][indice].squeeze()[1]
            positive_indices[i, j] = 1
            msk[j] = False
            
            # initialize the sample_score_margin
            sample_score_margin = -1
            num_trials = 0
            
            neg_labels_idx = all_labels_idx[msk]

            while ((sample_score_margin < 0) and (num_trials < max_num_trials)):
                 
                #randomly sample a negative label
                neg_idx = np.random.choice(neg_labels_idx, 1)[0]
                msk[neg_idx] = False
                neg_labels_idx = all_labels_idx[msk]
                
                num_trials += 1
                # calculate the score margin 
                sample_score_margin = 1 + input[i, neg_idx] - input[i, j] 
            
            if sample_score_margin < 0:
                # checks if no violating examples have been found 
                continue
            else: 
                loss_weight = np.log(np.floor((Y-1)/(num_trials)))
                L[i] = loss_weight
                negative_indices[i, neg_idx] = 1
        
        loss = L * (1-torch.sum(positive_indices*input, dim = 1) + torch.sum(negative_indices*input, dim = 1))
        
        ctx.save_for_backward(input, target)
        ctx.L = L
        ctx.positive_indices = positive_indices
        ctx.negative_indices = negative_indices
        
        return torch.sum(loss , dim = 0, keepdim = True)

    # This function has only a single output, so it gets only one gradient 
    @staticmethod
    def backward(ctx, grad_output):
        input, target = ctx.saved_tensors
        L = Variable(torch.unsqueeze(ctx.L, 1), requires_grad = False)

        positive_indices = Variable(ctx.positive_indices, requires_grad = False) 
        negative_indices = Variable(ctx.negative_indices, requires_grad = False)
        grad_input = grad_output * L * (negative_indices - positive_indices)

        return grad_input, None, None    

      
class WARPLoss(nn.Module): 
    def __init__(self, max_num_trials = None): 
        super(WARPLoss, self).__init__()
        self.max_num_trials = max_num_trials
        
    def forward(self, input, target): 
        return WARP.apply(input, target, self.max_num_trials)

## Define the model:

In [22]:
# Get cpu or gpu device for training.
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda:1 device


In [23]:
# Define model
class MF(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super(MF, self).__init__()
        
        self.user_factors = Parameter(torch.randn(n_users, n_factors))
        self.item_factors = Parameter(torch.randn(n_items, n_factors))
        self.user_biases = Parameter(torch.randn(n_users, 1))
        self.item_biases = Parameter(torch.randn(n_items, 1))
        
    def forward(self, user, item):
        pred = (self.user_biases[user] + self.item_biases[item]).squeeze()
        pred += (self.user_factors[user] * self.item_factors[item]).sum(dim=-1).squeeze()
        return pred

In [24]:
n_factors = 50
model = MF(n_users, n_items, n_factors).to(device)
print(model)

MF()


In [25]:
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, weight_decay=1e0)#lr=5e-3 weight_decay=1e-2
loss_fn = torch.nn.MSELoss()

## Train the model:

def train(train_data, model, loss_fn, optimizer):
    
    rows, cols = train_data.nonzero()
    p = np.random.permutation(len(rows))
    rows, cols = rows[p], cols[p]

    for row, col in zip(*(rows, cols)):

        # Turn data into tensors
        rating = torch.FloatTensor([train_data[row, col]]).squeeze()
        row = torch.LongTensor([row])
        cols = torch.LongTensor([col])
        
        # Compute prediction error
        prediction = model(row, col)
        loss = loss_fn(prediction, rating)

        # Set gradients to zero
        optimizer.zero_grad()
        
        # Backpropagation
        loss.backward()
        
        # Update the parameters
        optimizer.step()

In [26]:
def train_batch(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    for batch, (rows, cols) in enumerate(dataloader):
        rows_c, cols_c = rows.to(device), cols.to(device)

        # Compute prediction error
        rating = torch.tensor([dataloader.dataset.data[rows, cols]], device=device).squeeze()
        prediction = model(rows_c, cols_c).to(torch.float64)
        loss = loss_fn(prediction, rating)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 500 == 0:
            loss, current = loss.item(), batch * len(rows)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [27]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    train_batch(train_dataloader, model, loss_fn, optimizer)
    W = model.user_factors.mm(model.item_factors.T).cpu().detach().numpy()
    ub = model.user_biases.cpu().detach().numpy()
    ib = model.item_biases.cpu().detach().numpy()
    w = W + ub + ib.T
    print(f"\nMetric: HR@10 = {hr(test_data, test_dict, w, hm=10):.4f};\n")
print("Done!")

Epoch 1
-------------------------------
loss: 57.703419  [    0/99057]
loss: 0.990580  [32000/99057]
loss: 0.982432  [64000/99057]
loss: 0.986696  [96000/99057]

Metric: HR@10 = 0.1410;

Epoch 2
-------------------------------
loss: 0.985251  [    0/99057]
loss: 0.985638  [32000/99057]
loss: 0.983950  [64000/99057]
loss: 0.985674  [96000/99057]

Metric: HR@10 = 0.1400;

Epoch 3
-------------------------------
loss: 0.985696  [    0/99057]
loss: 0.984553  [32000/99057]
loss: 0.985555  [64000/99057]
loss: 0.986022  [96000/99057]

Metric: HR@10 = 0.1453;

Epoch 4
-------------------------------
loss: 0.985171  [    0/99057]
loss: 0.986430  [32000/99057]
loss: 0.986577  [64000/99057]
loss: 0.985985  [96000/99057]

Metric: HR@10 = 0.1485;

Epoch 5
-------------------------------
loss: 0.985141  [    0/99057]
loss: 0.985428  [32000/99057]
loss: 0.984548  [64000/99057]
loss: 0.986467  [96000/99057]

Metric: HR@10 = 0.1506;

Done!


## Evaluate the model:

In [30]:
W = model.user_factors.mm(model.item_factors.T).cpu().detach().numpy()
ub = model.user_biases.detach().cpu().numpy()
ib = model.item_biases.detach().cpu().numpy()
w = W + ub + ib.T

In [31]:
hr(test_data, test_dict, w, hm=10)

0.15058324496288442

### WARP model:

In [32]:
# Define model
class MFWARP(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super(MFWARP, self).__init__()
        
        self.user_factors = Parameter(torch.randn(n_users, n_factors))
        self.item_factors = Parameter(torch.randn(n_items, n_factors))
        self.user_biases = Parameter(torch.randn(n_users, 1))
        self.item_biases = Parameter(torch.randn(n_items, 1))
        
    def forward(self, users, items):
        pred = (self.user_biases[users] + self.item_biases[items].T)
        pred += (self.user_factors[users] @ self.item_factors[items].T)
        return pred

In [33]:
n_factors = 50
model = MFWARP(n_users, n_items, n_factors).to(device)
print(model)

MFWARP()


In [34]:
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, weight_decay=1e-2)
loss_fn = WARPLoss(20)

In [35]:
def train_batch_warp(train_data, model, loss_fn, optimizer, bs=64, show=True):
    user_idx = np.arange(train_data.shape[0])
    np.random.shuffle(user_idx)
    batches = np.array_split(user_idx, train_data.shape[0]//bs)
    cols = torch.arange(train_data.shape[1])
    cols_c = cols.to(device)
    
    for i, batch in enumerate(batches):
        rows = torch.tensor(batch)
        rows_c = rows.to(device)

        # Compute prediction error
        rating = torch.tensor([train_data[rows][:, cols]], device=device).squeeze()
        prediction = model(rows_c, cols_c).to(torch.float64)
        loss = loss_fn(prediction, rating)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if show and (i % 10 == 0):
            loss, current = loss.item(), i * len(rows)
            print(f"loss: {loss:>7f}  [{current:>5d}/{train_data.shape[0]:>5d}]")

In [36]:
epochs = 300
for epoch in range(epochs):
    train_batch_warp(train_data, model, loss_fn, optimizer, 64, show=False)
    W = model.user_factors.mm(model.item_factors.T).cpu().detach().numpy()
    ub = model.user_biases.cpu().detach().numpy()
    ib = model.item_biases.cpu().detach().numpy()
    w = W + ub + ib.T
    
    if epoch % 20 == 0:   
        print(f"Epoch {epoch + 1}: Metric: HR@10 = {hr(test_data, test_dict, w, hm=10):.4f};")
print("Done!")

Epoch 1: Metric: HR@10 = 0.0148;
Epoch 21: Metric: HR@10 = 0.0212;
Epoch 41: Metric: HR@10 = 0.0498;
Epoch 61: Metric: HR@10 = 0.0742;
Epoch 81: Metric: HR@10 = 0.0933;
Epoch 101: Metric: HR@10 = 0.1124;
Epoch 121: Metric: HR@10 = 0.1188;
Epoch 141: Metric: HR@10 = 0.1251;
Epoch 161: Metric: HR@10 = 0.1400;
Epoch 181: Metric: HR@10 = 0.1453;
Epoch 201: Metric: HR@10 = 0.1485;
Epoch 221: Metric: HR@10 = 0.1485;
Epoch 241: Metric: HR@10 = 0.1654;
Epoch 261: Metric: HR@10 = 0.1697;
Epoch 281: Metric: HR@10 = 0.1803;
Epoch 301: Metric: HR@10 = 0.1919;
Epoch 321: Metric: HR@10 = 0.1813;
Epoch 341: Metric: HR@10 = 0.1792;
Epoch 361: Metric: HR@10 = 0.2068;
Epoch 381: Metric: HR@10 = 0.1962;
Epoch 401: Metric: HR@10 = 0.2004;
Epoch 421: Metric: HR@10 = 0.2015;
Epoch 441: Metric: HR@10 = 0.1888;


KeyboardInterrupt: 